In [51]:
import time
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch_geometric.nn import to_hetero
from torch_geometric.loader import DataLoader, NeighborLoader
from sklearn.model_selection import train_test_split

from clauserec import ClauseRec
from dataset import DimacsDataset
from validation import validate

In [52]:
root_dir = "../data/"
data_dir = "../../solved/tr/"
NMAX = 1000

dataset = DimacsDataset(root_dir, data_dir, nmax=NMAX)

train_set, test_set = train_test_split(dataset, test_size=0.1, random_state=42)

print(len(train_set))
print(len(test_set))

The train file J6033_3.RCP.train has been created.: 100%|██████████| 185/185 [01:02<00:00,  2.98it/s] 
Processing...
Done!


148
19
18


In [3]:
# Instantiate a homogeneous GNN
model = ClauseRec(hidden_channels=64) 
# Convert it to Heterogeneous to work with our data
model = to_hetero(model, dataset.getMetadata(),aggr='sum')

In [4]:
train_loader = DataLoader(train_set,batch_size=4,shuffle=True)
test_loader = DataLoader(test_set,batch_size=4,shuffle=True)

In [5]:
train_loader

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
optimizer = torch.optim.Adam(model.parameters(), lr=0.5)
#criterion = torch.nn.BCELoss()
criterion = torch.nn.MSELoss()
model = model.to(device)

In [7]:
## TRAIN ##

n_epochs = 100 
K=3
loss_vals=  []

for epoch in (pbar := tqdm(range(n_epochs))):
    epoch_loss= []
    precisionK = recallK = total = total_loss =0
    for batch in train_loader:
        
        model.train()
        optimizer.zero_grad()
        batch = batch.to(device)
                                
        pred = model(batch.x_dict, batch.edge_index_dict)
        loss = criterion(pred['clauses'], batch['clauses'].y)
                
        loss.backward()
        optimizer.step()
        total_loss += float(loss)
        epoch_loss.append(loss.item())
        
    avg_loss = sum(epoch_loss)/len(epoch_loss)
        
    with torch.no_grad():
        model.eval()
        for vbatch in test_loader:
            vbatch = vbatch.to(device)
            pK, rK = validate(vbatch, model, num_iterations=100, K=K)
            precisionK += pK
            recallK += rK
            total += 1
            
    avg_precisionK = precisionK / total
    avg_recallK = recallK / total
    
    loss_vals.append(avg_loss)

    pbar.set_description(f"Train Loss = {avg_loss}, Test P@K={K} = {avg_precisionK}, Test R@K={K} = {avg_recallK}")
    
plt.plot(np.linspace(1, n_epochs, n_epochs).astype(int), loss_vals)

Train Loss = 16824.596165060997, Test P@K=3 = 0.8099999999999999, Test R@K=3 = 0.24299999999999997:  11%|█         | 11/100 [2:08:54<23:04:06, 933.10s/it] 

In [35]:
model = ClauseRec(hidden_channels=128)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.load_state_dict(torch.load(os.path.join("../data/","models","Regressor128.pt")),strict=False)
model = to_hetero(model, dataset.getMetadata(),aggr='sum')

In [50]:
## VALIDATION ##
K = 1
I = 1000
N_conflicts = 10

precisionK = recallK = total = 0
test_loader = DataLoader(test_set,batch_size=4,shuffle=True)

model.eval()
model = model.to(device)
for index, batch in (pbar := tqdm(enumerate(test_loader))):
    batch = batch.to(device)
    pK, rK = validate(batch,model,I,N_conflicts,K)
    precisionK += pK
    recallK += rK
    total += 1
        
print(f"Final Precision@K is: {precisionK / total}")
print(f"Final Recall@K is: {recallK / total}")

5it [00:34,  6.81s/it]

Final Precision@K is: 0.8726
Final Recall@K is: 0.08726444444444377


In [4]:
#torch.save(model, os.path.join(dataset.root,"models",time.strftime("%Y%m%d-%H%M%S") + ".pt"))
torch.save(model.state_dict(), os.path.join(dataset.root,"models",time.strftime("%Y%m%d-%H%M%S") + ".pt"))

In [ ]:
#model = torch.load(os.path.join(dataset.root,"models","20240410-091906.pt"))
""" model = ClauseRec(hidden_channels=64)
model.load_state_dict(torch.load(os.path.join(dataset.root,"models","20240430-121222.pt")),strict=False)
model = to_hetero(model, dataset.getMetadata(),aggr='sum') """

' model = ClauseRec(hidden_channels=64)\nmodel.load_state_dict(torch.load(os.path.join(dataset.root,"models","20240430-121222.pt")),strict=False)\nmodel = to_hetero(model, dataset.getMetadata(),aggr=\'sum\') '

In [ ]:
""" test_loader = DataLoader(dataset,batch_size=64,shuffle=False)
model.eval()

for batch in tqdm(test_loader):
    model(batch.x_dict, batch.edge_index_dict) """

' test_loader = DataLoader(dataset,batch_size=64,shuffle=False)\nmodel.eval()\n\nfor batch in tqdm(test_loader):\n    model(batch.x_dict, batch.edge_index_dict) '